In [1]:
import os
os.chdir("../")

In [2]:
from src.utils import *

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
import openai

In [4]:
#load the data
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["data/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
# Checking the length of the document

print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: fb5d5057-046c-45bf-9fc9-2b93689712f3
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [6]:
# Building up a RAG pipeline

from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [7]:
document.text

'PAGE 1Founder, DeepLearning.AICollected Insights\nfrom Andrew Ng\nHow to \nBuild\nYour\nCareer\nin AIA Simple Guide\n\n\nPAGE 2"AI is the new \nelectricity. It will \ntransform and improve \nall areas of human life."\nAndrew Ng\n\nPAGE 3Table of \nContentsIntroduction: Coding AI is the New Literacy.\nChapter 1: Three Steps to Career Growth.\nChapter 2: Learning Technical Skills for a \nPromising AI Career.\nChapter 3: Should You Learn Math to Get a Job \nin AI?\nChapter 4: Scoping Successful AI Projects.\nChapter 5: Finding Projects that Complement \nYour Career Goals.\nChapter 6: Building a Portfolio of Projects that \nShows Skill Progression.\nChapter 7: A Simple Framework for Starting Your AI \nJob Search.\nChapter 8: Using Informational Interviews to Find \nthe Right Job.\nChapter 9: Finding the Right AI Job for You.\nChapter 10: Keys to Building a Career in AI.\nChapter 11: Overcoming Imposter Syndrome.\nFinal Thoughts: Make Every Day Count.LEARNING\nPROJECTS\nJOB\n\nPAGE 4Coding

In [8]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

# 1 using service context
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [9]:
# we will now run a query engine
query_engine = index.as_query_engine()

In [10]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


When finding projects to build your experience, there are several steps you can take. First, you can join existing projects by asking to join someone else's project if they have an idea. Additionally, you can continue reading, taking courses, and talking to domain experts to come up with new ideas. It is also helpful to focus on a specific application area where machine learning has not yet been applied. This can give you the opportunity to explore unique and creative applications that no one else has done yet. Finally, you can develop a side hustle or personal project that may or may not develop into something bigger, as this can stir your creative juices and strengthen bonds with collaborators.


In [11]:
# Evaluation setup using TruLens

In [12]:
eval_questions = []
with open('data/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [13]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [14]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'What is the right AI job for me?']

In [15]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [16]:
from src.utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [17]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [18]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [19]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_61f1a847524cd04ece4cdc67de90d079,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_61f1a847524cd04ece4...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-17T23:08:20.258050"", ""...",2024-01-17T23:08:26.416950,0.9,0.6,1.000,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",6,2184,0.003360
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a8e11c37e3e0547eee60b56a20554d3b,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_a8e11c37e3e0547eee6...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-17T23:08:26.755657"", ""...",2024-01-17T23:08:29.360719,0.9,0.0,1.000,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,2,1712,0.002612
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_11d9c1995c9f7024ee080b46d4753a8a,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_11d9c1995c9f7024ee0...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-17T23:08:29.654651"", ""...",2024-01-17T23:08:33.913555,0.9,0.1,0.250,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,4,1706,0.002599
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_612c1b1e07bb49582bc4eee9252a7dee,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_612c1b1e07bb49582bc...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-17T23:08:34.289604"", ""...",2024-01-17T23:08:38.542962,1.0,0.6,1.000,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,4,1680,0.002563
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5f90448b2d3e8211927858394fb4c123,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_5f90448b2d3e8211927...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-17T23:08:38.866722"", ""...",2024-01-17T23:08:42.591028,0.9,0.3,0.625,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,3,1669,0.002541


In [20]:
feedback

['Groundedness', 'Context Relevance', 'Answer Relevance']

In [21]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.29.48:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [22]:
# Advanced RAG pipeline

In [23]:
## 2 Sentence Window retrieval

In [24]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [25]:
from src.utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [26]:
from src.utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [27]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, it is important to first identify and scope the project. Consider your career goals and choose a project that complements them. Ensure that the project is responsible, ethical, and beneficial to people. As you progress in your career, aim for projects that grow in scope, complexity, and impact over time. Building a portfolio of projects that shows skill progression can also be helpful. Additionally, there are resources available in the book that provide guidance on starting your AI job search and finding the right AI job for you.


In [28]:
# tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [29]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)

In [30]:
tru.get_leaderboard(app_ids=["Direct Query Engine" , "Sentence Window Query Engine"])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.767857,0.350000,0.888889,3.272727,0.000850
Direct Query Engine,0.715606,0.263636,0.900000,3.272727,0.003059


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.29.48:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [32]:
# Auto Merging retrieval

In [33]:
from src.utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [34]:
from src.utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [35]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: ce3c2ac8-3dfd-41f2-9920-a2a5bd2ef73d.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 272eacd0-f132-430d-aae5-1c2af917e65f.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones. This progression will demonstrate your growth and development over time. Additionally, effective communication is crucial. You should be able to explain your thinking and articulate the value of your work to others. This will help build trust and increase your chances of securing resources for larger projects.


In [36]:
# tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [37]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)

> Merging 2 nodes into parent node.
> Parent node id: cf233675-66c2-4632-8f97-b83c15c4af43.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 0809d528-2341-4993-ba89-194a76fb9205.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 3 nodes into parent node.
> Parent node id: 5d9391e5-4318-4f1c-ab63-c93cdd6d6cc2.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...

> Merging 1 nodes into parent node.
> Parent node id: e265ad3f-8375-4e1d-9b20-d336c529ad81.
> Parent node text: PAGE 35Keys to Building a Career in AI CHAPTER 10
The path to career success in AI is more comple...

> Merging 2 nodes into parent node.
> Parent node id: aca7b454-1797-4837-8163-35fbe1006f00.
> Parent node text: PAGE 36Keys to Building a Career

In [38]:
tru.get_leaderboard(app_ids=["Automerging Query Engine"])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.915,0.591667,0.9,2.363636,0.000842


In [39]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.29.48:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>